In [ ]:
# Librerie base
import numpy as np
import pandas as pd

# Visualizzazione
import matplotlib.pyplot as plt  
import seaborn as sns

# Modelli statistici
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

# Libreria ISLP (Statistical Learning)
from ISLP import load_data
from ISLP.models import (ModelSpec as MS ,summarize, poly)

# Simple Linear Regression
We will use the Boston housing dataset, contained in the ISLP package. The Boston dataset records the medv (average house value) for 506 neighbourhoods around Boston.

In [ ]:
# carico il dataset e stampo l'head
f1 = pd.read_csv("f1_pitstops_2018_2024.csv", encoding='utf-8')

In [ ]:
f1.columns

In [ ]:
#look at the first 5 rows in the datasets
f1.head()

In [ ]:
print(f1.describe())

In [ ]:
# Controllo i dati nulli
columnsWithNulls=f1.isnull().sum().sort_values(ascending=False)
columnsWithNulls=columnsWithNulls[columnsWithNulls>0]
print(columnsWithNulls)

In [ ]:
# Controllo in percentuale quanti dati mancano così da capire come trattarli, pongo una soglia di eliminazione del regressore nel caso di +30% di dati mancanti
missing_pct = f1.isnull().mean() * 100
missing_only = missing_pct[missing_pct > 0].sort_values(ascending=False)
print(missing_only)
sns.barplot(x=missing_only.values, y=missing_only.index)
plt.title('Percentuale di valori nulli per colonna')
plt.xlabel('% Missing')
plt.ylabel('Colonna')
plt.show()

In [ ]:
#droppiamo le righe con i valori nulli e le colonne con più del 30% di dati nulli
f1.dropna(axis=0, inplace=True)
f1.drop(columns=missing_only[missing_only>30].index, inplace=True)
f1.isnull().sum()

## Data processing

In [ ]:
# Seleziona le colonne numeriche
numeric_f1 = f1.select_dtypes(include=[np.number])

# Controllo che ci siano almeno 4 variabili numeriche per evitare visualizzazioni poco utili
if numeric_f1.shape[1] >= 4:
    # Calcola la matrice di correlazione
    corr = numeric_f1.corr()
    
    # Imposta la figura
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        corr,
        annot=True,
        fmt='.2f',
        cmap='coolwarm',
        square=True,
        linewidths=.5,
        cbar_kws={"shrink": .75}
    )
    plt.title('Correlation Heatmap of Numeric Features')
    plt.tight_layout()
    plt.show()
else:
    print('Not enough numeric columns for a meaningful correlation heatmap')

In [ ]:
columns_to_drop = ['Position Changes', 'Total Pit Stops', 'Fast Lap Attempts', 'Lap Time Variation', 'Air_Temp_C', 'AvgPitStopTime']
f1.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# Seleziona le colonne numeriche
numeric_f1 = f1.select_dtypes(include=[np.number])

# Controllo che ci siano almeno 4 variabili numeriche per evitare visualizzazioni poco utili
if numeric_f1.shape[1] >= 4:
    # Calcola la matrice di correlazione
    corr = numeric_f1.corr()
    
    # Imposta la figura
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        corr,
        annot=True,
        fmt='.2f',
        cmap='coolwarm',
        square=True,
        linewidths=.5,
        cbar_kws={"shrink": .75}
    )
    plt.title('Correlation Heatmap of Numeric Features')
    plt.tight_layout()
    plt.show()
else:
    print('Not enough numeric columns for a meaningful correlation heatmap')

In [ ]:
numeric_cols = f1.select_dtypes(include='number').columns
x_vars = [col for col in numeric_cols if col != 'Tire Usage Aggression']

# Controlla se ci sono abbastanza variabili da visualizzare
n = len(x_vars)
if n > 0:
    # Imposta il layout della figura
    fig, axes = plt.subplots(1, n, figsize=(5 * n, 5), sharey=True)

    # Scatterplot di ogni variabile rispetto al Driver Aggression Score
    for i, var in enumerate(x_vars):
        sns.scatterplot(data=f1, x=var, y='Tire Usage Aggression', ax=axes[i])
        axes[i].set_title(f'{var} vs Aggression')
        axes[i].set_xlabel(var)

    axes[0].set_ylabel('Tire Usage Aggression')
    plt.tight_layout()
    plt.show()
else:
    print('Nessuna variabile numerica disponibile per il confronto con Driver Aggression Score')

## Linear model

In [ ]:
numeric_cols

### Choose the first 3 variables that are more promising

In [ ]:
# Target variable
target = 'Tire Usage Aggression'

results = []

# Looping every possible variable
for col in numeric_cols:
    X = sm.add_constant(f1[[col]])
    y = f1[target]
    
    model = sm.OLS(y, X).fit()
    
    results.append({
        'variable': col,
        'r_squared': model.rsquared,
        'p_value': model.pvalues[col]
    })

# Transforming data frame
results_df = pd.DataFrame(results)

# Ordering
top_r2_pval = results_df.sort_values(by='p_value', ascending=True).head(4)
worse_r2_pval = results_df.sort_values(by='p_value', ascending=False).head(3)


# Output
print("VARIABILI CON R² PIU' ALTO E p-value PIU BASSO:")
print(top_r2_pval)

print("\n\nVARIABILI CON R² PIU' BASSO E p-value PIU ALTO:\n")
print(worse_r2_pval)

### Fit Linear Model: mdev = b0 + b1*TotalPitStops + e

In [ ]:
X = pd.DataFrame({'intercept': np.ones(f1.shape[0]), 'TotalPitStops': f1['TotalPitStops']})
X[:4]

In [ ]:
y = f1[target]
model = sm.OLS(y, X) #function to fit a simple linear regression. Here we define the model
results = model.fit() #here we fit the model

In [ ]:
print(results.summary())

significant_vars = results.pvalues[results.pvalues < 0.05]
print("\nVariabili significative (p-value < 0.05):")
print(significant_vars)

insignificant_vars = results.pvalues[results.pvalues >= 0.05]
print("\nVariabili insignificanti (p-value >= 0.05):")
print(insignificant_vars)

### Creating the input matrix using ModelSpec of ISLP package

In [ ]:
model = MS(['TotalPitStops'])
model = model.fit(f1) 
X = model.transform(f1)
X[:4]

In [ ]:
sns.histplot(y, kde=True)

plt.xlim(y.quantile(0.00), y.quantile(0.95))

plt.title('Distribution of Tire Usage Aggression (Up to 95th Percentile)')
plt.show()

In [ ]:
sns.histplot(f1['TotalPitStops'], kde=True)

plt.xticks(f1['TotalPitStops'].unique(), rotation=0)

plt.title('Distribution of Total Pit Stops')
plt.show()

In [ ]:
# Get predictions on new input
new_df = pd.DataFrame({'TotalPitStops': [2, 3, 4, 5, 6, 7]})
new_X = model.transform(new_df)  # Aggiungi una colonna di 1 per l'intercetta
new_X

In [ ]:
New_X = sm.add_constant(new_df)
new_predictions = results.get_prediction(new_X)
predicted_means = new_predictions.predicted_mean
print(predicted_means)

In [ ]:
#Confidence interval
new_predictions.conf_int(alpha=0.05)

In [ ]:
intercept = results.params.iloc[0]  # Intercetta (b)
slope = results.params.iloc[1]  # Pendenza (m)

formula = f"y = {slope:.4f} * TotalPitStops + {intercept:.4f}"

print("FORMULA del modello di regressione:", formula)

In [ ]:
def abline(ax, b, m, *args, **kwargs):
    "Aggiungi una retta con pendenza m e intercetta b su ax"
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim, *args, **kwargs)

ax = plt.gca()
ax.scatter(f1['TotalPitStops'], f1['Tire Usage Aggression'], alpha=0.5)

abline(ax, intercept, slope, 'r--', linewidth=3)

plt.show()

In [ ]:
# Diagnostics plot (We observe non linearity)
fig, ax = plt.subplots(figsize=(8, 8)) 
ax.scatter(results.fittedvalues, results.resid)  # Scatter plot of fitted values vs residuals
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--')  


In [ ]:
# Compute leverage statistics
infl = results.get_influence()
ax = plt.subplots(figsize=(8,8))[1]
ax.scatter(np.arange(X.shape[0]), infl.hat_matrix_diag)
ax.set_xlabel('Index')
ax.set_ylabel('Leverage')
np.argmax(infl.hat_matrix_diag)

# Multiple Linear Regression

In [ ]:
# y = b*X + e (perform a regression using all of the predictors)
terms = numeric_cols.drop('Tire Usage Aggression')
print(terms)

In [ ]:
X = MS(terms).fit_transform(f1)
model = sm.OLS(y, X)
results = model.fit()
summarize(results)

In [ ]:
significant_vars = results.pvalues[results.pvalues < 0.05]
print("\nVariabili significative (p-value < 0.05):")
print(significant_vars)

insignificant_vars = results.pvalues[results.pvalues >= 0.05]
print("\nVariabili insignificanti (p-value >= 0.05):")
print(insignificant_vars)

In [ ]:
# getting goodness of fit
print("R2", results.rsquared)
print("RSE", np.sqrt(results.scale))

In [ ]:
minus_var = terms.drop(['Season', 'Round', 'Track_Temp_C', 'Humidity_%', 'Stint', 'Stint Length'])
Xma = MS(minus_var).fit_transform(f1)
model1 = sm.OLS(y, Xma)
results1 = model1.fit()
summarize(results1)

In [ ]:
significant_vars = results1.pvalues[results1.pvalues < 0.05]
print("\nVariabili significative (p-value < 0.05):")
print(significant_vars)

insignificant_vars = results1.pvalues[results1.pvalues >= 0.05]
print("\nVariabili insignificanti (p-value >= 0.05):")
print(insignificant_vars)

In [ ]:
print("R2", results1.rsquared)
print("RSE", np.sqrt(results1.scale))

## Interaction Terms

In [ ]:
# to introduce and interactions terms use a python tuple
X = MS(['lstat', 'age', ('lstat', 'age')]).fit_transform(Boston)
model2 = sm.OLS(y, X)
summarize(model2.fit())

## Non-linear Transformations of the Predictors

In [ ]:
# poly() function present in the package ISLP specifies that columns representing polynomial functions of its first argument are added to the model matrix
X = MS([poly('lstat', degree=2), 'age']).fit_transform(Boston)
model3 = sm.OLS(y, X)
results3 = model3.fit()
summarize(results3)

## Qualitative predictors
Based on the Carseats dataset present in the package ISLP, will attempt to predict Sales (child car seat sales) in 400 locations based
on a number of predictors.

The Carseats data includes the qualitative predictor "ShelveLoc," which indicates the quality of the shelving location with three possible values: Bad, Medium, and Good.

In general for qualitative predictor the ModelSpec() generates one-hot encoding of the categorical variables automatically

In [ ]:
Carseats = load_data('Carseats')
Carseats.columns

In [ ]:
allvars = list(Carseats.columns.drop('Sales'))
y = Carseats['Sales']
final = allvars + [('Income', 'Advertising'),
('Price', 'Age')]
X = MS(final).fit_transform(Carseats)
model = sm.OLS(y, X)
summarize(model.fit())

# Cross Validation

### Validation set approach

In [ ]:
from functools import partial
from sklearn.model_selection import (cross_validate ,KFold ,ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm #a wrapper that enables us to easily use the cross-validation tools of sklearn with models fit by statsmodels
from sklearn.model_selection import train_test_split

In [ ]:
Auto = load_data('Auto')
Auto_train , Auto_valid = train_test_split(Auto, test_size=196, random_state=0) # random_state is needed for reproducible result across run

In [ ]:
#fit a linear regression model
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train , X_train)
results = model.fit()

In [ ]:
# evaluate the model using the MSE on the validation data
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

## Leave One-out Cross validation
The sklearn_sm() class takes a statsmodels model as its first argument. It also accepts two optional arguments: model_str for specifying a formula, and model_args, which is a dictionary containing additional arguments for fitting the model.

In [ ]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']

# This is a LOOCV because cv use the number of sample in our dataset, that is Auto.shape[0], making sure that each sample is use as a test set
cv_results = cross_validate(hp_model ,X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

## K-Fold cross validation

In [ ]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
kf = KFold(n_splits=10, shuffle=True, random_state=1)
cv_results = cross_validate(hp_model, X, Y, cv=kf)
cv_err = np.mean(cv_results['test_score'])
cv_err

# Exercise
Train and compare the performance of different cross-validation methods to identify the best model for polynomial regression with varying degrees using the Auto dataset.

https://scikit-learn.org/stable/

Fine